In [1]:
pip install mysql.connector

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     --------------------------------------- 0.1/11.9 MB 469.7 kB/s eta 0:00:26
     --------------------------------------- 0.1/11.9 MB 656.4 kB/s eta 0:00:18
     --------------------------------------- 0.1/11.9 MB 778.5 kB/s eta 0:00:16
      --------------------------------------- 0.3/11.9 MB 1.1 MB/s eta 0:00:11
     - -------------------------------------- 0.4/11.9 MB 1.3 MB/s eta 0:00:09
     - -------------------------------------- 0.5/11.9 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.6/11.9 MB 1.7 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/11.9 MB 1.7 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/11.9 MB 1.8 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/11.9 MB 1.9 MB/s eta


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install ahpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import mysql.connector
import ahpy

# Connect to the MySQL database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='electric_car'
)
cursor = connection.cursor()

# Fetch criteria and alternatives from the database
cursor.execute("SELECT DISTINCT criterion1 FROM criteria_comparisons UNION SELECT DISTINCT criterion2 FROM criteria_comparisons")
criteria_rows = cursor.fetchall()
print("Criteria Rows:", criteria_rows)  # Debugging: print criteria rows

criterias = [row[0] for row in criteria_rows]

cursor.execute("SELECT DISTINCT alternative1 FROM alternative_comparisons UNION SELECT DISTINCT alternative2 FROM alternative_comparisons")
alternative_rows = cursor.fetchall()
print("Alternative Rows:", alternative_rows)  # Debugging: print alternative rows

alternatives = [row[0] for row in alternative_rows]

# Fetch criteria comparisons from the database
cursor.execute("SELECT criterion1, criterion2, value FROM criteria_comparisons")
criteria_comparisons = {(row[0], row[1]): row[2] for row in cursor.fetchall()}

# Fetch alternative comparisons from the database
cursor.execute("SELECT criterion, alternative1, alternative2, value FROM alternative_comparisons")
alternative_comparisons = {}
for row in cursor.fetchall():
    criterion, alternative1, alternative2, value = row
    if criterion not in alternative_comparisons:
        alternative_comparisons[criterion] = {}
    alternative_comparisons[criterion][(alternative1, alternative2)] = value

# Close the database connection
connection.close()

# Create the AHP model for criteria
criteria_ahp = ahpy.Compare('Criteria', criteria_comparisons, precision=3)

# Create the AHP models for alternatives with respect to each criterion
alternative_ahp = {}
for criterion in criterias:
    alternative_ahp[criterion] = ahpy.Compare(criterion, alternative_comparisons[criterion], precision=3)

# Combine the criteria and alternative priorities to calculate the overall scores
overall_scores = {}
for alt in alternatives:
    overall_scores[alt] = sum(criteria_ahp.target_weights[crit] * alternative_ahp[crit].target_weights[alt] for crit in criterias)


Criteria Rows: [('harga',), ('kinerja',), ('keamanan',), ('kapasitas baterai',), ('kenyamanan',), ('tingkat penjualan',)]
Alternative Rows: [('Ioniq',), ('Air EV',), ('Nissan Leaf',), ('Lexus',), ('Kia',)]


In [11]:
# Insert results into the database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='electric_car'
)
cursor = connection.cursor()

cursor.execute("DELETE FROM ahp_results")
for idx, (alternative, score) in enumerate(sorted(overall_scores.items(), key=lambda item: item[1], reverse=True), start=1):
    cursor.execute("INSERT INTO ahp_results (id, alternative, score) VALUES (%s, %s, %s)", (idx, alternative, float(score)))

# Commit the transaction
connection.commit()
connection.close()

# Print overall scores for debugging
print("Overall Scores:", overall_scores)

Overall Scores: {'Ioniq': 0.225103, 'Air EV': 0.208314, 'Nissan Leaf': 0.116361, 'Lexus': 0.19560900000000003, 'Kia': 0.255373}
